Allison Forte

DSC 540

Project: Milestone 4

August 10, 2022

# Connecting to an API/Pulling in the Data and Cleaning/Formatting
Perform at least 5 data transformation and/or cleansing steps to your API data. Label each transformation (Step #1, Step #2, etc.) in your code and describe what it is doing in 1-2 sentences.

In [268]:
#  Install libraries

import urllib.request, urllib.parse, urllib.error
import json
import requests


# Set up connection to API using API key

with open('/Users/allison.forte/Downloads/API keys') as f:
    api_keys = f.readlines()
    api_key = api_keys[9]


def search_currency(currency):
    try:
        code = currency
        url = 'https://v6.exchangerate-api.com/v6/' + api_key + 'latest/' + code
        print(f'Retrieving data for "{currency}".')  # Let the user know the search is working
        response = requests.get(url)  # Accessing the API
        data = response.json()  # Data received from the API

        if data['result'] == 'success':
            return(data)
    
        else:
            print("Error encountered")  # Informs user of error
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")  # Displays error if the request is not successful

# Step 1: Define countries we want exchange rates for

Pull list of country codes from website data to define which countries we are looking for and avoid duplicate entries in list

In [269]:
# Cleaned df- steps from milestone 2 (for reference only, not used in this milestone)

import numpy as np
import pandas as pd


def monthToNum(shortMonth):
    return {
            'Jan': 1,
            'Feb': 2,
            'Mar': 3,
            'Apr': 4,
            'May': 5,
            'Jun': 6,
            'Jul': 7,
            'Aug': 8,
            'Sep': 9, 
            'Oct': 10,
            'Nov': 11,
            'Dec': 12}[shortMonth]


file = pd.read_csv('/Users/allison.forte/Documents/540 Assignments/tech_fundings.csv')
df = pd.DataFrame(data=file)  # Create df
df.rename(columns = {'Region':'Company_HQ', 'Vertical':'Category', 'Funding Amount (USD)':'Funding_amt', 
                     'Funding Date':'Funding_date'}, inplace = True)  # Rename columns
df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)  # Drop missing data
df['Funding_amt'] = df['Funding_amt'].replace(['Unknown'],'0')  # Fix missing data
df = df.drop_duplicates(subset='Website', keep='first', inplace=False)  # Drop duplicates
df[['Funding_month', 'Funding_year']] = df['Funding_date'].str.split('-', expand=True)  # Split date to 2 columns
months = (df['Funding_month'])
month_num  = []
for m in months:
    month_num.append(monthToNum(m))
df['month_num'] = month_num  # Add month number to column
df['Funding_amt'] = df['Funding_amt'].astype(float)  # Change data type
df['Funding_year'] = df['Funding_year'].astype(float)  # Change data type
df = df[(df['Funding_amt'] <70000000) & (df['Funding_amt'] > 0)]  # Remove outliers

In [270]:
# Cleaned web site data from milestone 2, used to get list of country codes for API

import lxml
import html5lib
import pandas as pd

url1 = 'https://www.iban.com/exchange-rates'
dfs1 = pd.read_html(url1)
df_convert = dfs1[0]

# Add one row with EUR conversion rate since the table is based on a EUR conversion rate
new_row = {'Currency':'EUR', 'Currency Name': 'Euro', 'Exchange Rate = 1 EUR':1, 'Convert':'NaN'}
df_convert = df_convert.append(new_row, ignore_index=True)

# Second source needed for the exchange rates- original data contained 'number' that was not the exchange rate
url2 = 'https://www.iban.com/currency-codes'
dfs2 = pd.read_html(url2)
df_codes = dfs2[0]

df_convert.rename(columns = {'Currency':'Currency_code', 'Currency Name':'Currency_name', 
                             'Exchange Rate = 1 EUR':'Exchange_rate_1EUR'}, inplace = True) # Change headers in df_convert
df_codes.rename(columns = {'Currency':'Currency_name', 'Code':'Currency_code'}, inplace = True) # Change headers in df_codes
df_convert.drop(columns = ['Convert'], inplace = True)  # Drop unneeded rows from df_convert
df_codes.drop(columns=['Number'], inplace = True)  # Drop unneeded rows from df_codes
df_codes.dropna(axis = 0, how = 'any', thresh = None, subset = None, inplace = True)  # Fixing missing data in df_codes
final_df = df_codes.merge(df_convert, on = 'Currency_code', suffixes=('', '_drop'))  # Merge the 2 data sets
final_df.drop(columns=['Currency_name_drop'], inplace = True)  # Drop the second currency_name column

In [271]:
#  Pull the currency codes only and remove duplicates

code = final_df['Currency_code']
codes = code.drop_duplicates(keep='first', inplace=False)

# Step 2: Retrieve the necessary exchange rate data

Retrieve JSON data from API for each country listed in CSV file

In [272]:
responses = []   
for c in codes:
    responses.append(search_currency(c))

Retrieving data for "EUR".


ConnectionError: HTTPSConnectionPool(host='v6.exchangerate-api.com', port=443): Max retries exceeded with url: /v6/c3bde5ac9f53549b21a99cd9latest/EUR (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8edf06be50>: Failed to establish a new connection: [Errno 60] Operation timed out'))

# Step 3: Remove unnecessary data by adding specific key to a new dictionary

We only need the 'conversion_rates' from the returned JSON data for each. Note the first entry of each returned result is the base currency.

In [ ]:
rates = {}
p=0
for r in responses:
    rates[p] = r['conversion_rates']
    p=p+1

# Step 4: Create a df with all results with currency_code as the row titles

To make these results easy to read and work with, create a df with the information

In [ ]:
rates_df = pd.DataFrame.from_dict(rates, orient = 'columns')
rates_df.head()

# Step 5: Change headers to make the df usable


In [ ]:
names = {0: 'EUR', 1: 'USD', 2: 'AUD', 3:'INR', 4:'NOK', 5:'BRL', 6: 'BGN', 7: 'CAD', 8: 'CNY',
         9: 'NZD', 10: 'HRK', 11: 'CZK', 12: 'DKK', 13: 'GBP', 14: 'HKD', 15: 'HUF', 16: 'ISK',
         17: 'IDR', 18: 'ILS', 19: 'JPY', 20: 'KRW', 21: 'ZAR', 22: 'CHF', 23: 'MYR', 24: 'MXN', 
         25: 'PHP', 26: 'PLN', 27: 'RON', 28: 'SGD', 29: 'SEK', 30: 'THB', 31: 'TRY'}
rates_df.rename(columns=names, inplace = True)
rates_df

# Step 6: Check data type and correct if needed
Confirm all exchange rates are numbers rather than strings, fix if needed.

In [ ]:
print('The starting data types are listed below:')
print(rates_df.dtypes)

All exchange rates are floats. There is no need for corrections.